In [15]:
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin


from collections import Counter , defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer


# To ignore warnings
import warnings
warnings.filterwarnings("ignore")



In [14]:
from iteration_utilities import duplicates, unique_everseen

print("get only duplicate value : ", list(duplicates([1,1,2,1,2,3,4,2])))

print("all unique value         : ", list(unique_everseen([1,1,2,1,2,3,4,2])))

get only duplicate value :  [1, 1, 2, 2]
all unique value         :  [1, 2, 3, 4]


# Functions

# Impute for missing value(Numberical : mean, categorical :mode(0))

In [4]:

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0] if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [10]:
data = [
    ['a', 1, 2],
    ['b', 1, 1],
    ['b', 2, 2],
    [np.nan, np.nan, np.nan]]

df5 = pd.DataFrame(data , columns=["A", "B", "C"])


In [8]:
df5.loc[:,"D"] = ["Raka" ,"rakesh", "Raka", np.nan]
df5

,A,B,C,D
0,a,1.0,2.0,Raka
1,b,1.0,1.0,rakesh
2,b,2.0,2.0,Raka
3,NaN,NaN,NaN,NaN


In [9]:
df4 = DataFrameImputer().fit_transform(df5)
df4

,A,B,C,D
0,a,1.000000,2.000000,Raka
1,b,1.000000,1.000000,rakesh
2,b,2.000000,2.000000,Raka
3,b,1.333333,1.666667,Raka


In [99]:
from pandas.api.types import is_numeric_dtype
def remove_outlier(dataframe):
    low = .05
    high = .95
    quant_df = dataframe.quantile([low, high])
    for name in list(dataframe.columns):
        if is_numeric_dtype(dataframe[name]):
            dataframe = dataframe[(dataframe[name] > quant_df.loc[low, name]) & (dataframe[name] < quant_df.loc[high, name])]
    return dataframe


In [100]:
	import numpy as np
	from pandas.api.types import is_numeric_dtype
	np.random.seed(42)
	age = np.random.randint(20,100,50)
	name = ['name'+str(i) for i in range(50)]
	address = ['address'+str(i) for i in range(50)]
	df1 = pd.DataFrame(data={'age':age, 'name':name, 'address':address})
	df1.head(2)

,age,name,address
0,71,name0,address0
1,34,name1,address1


In [1]:
t1 = [.01, 24, 81, 70, 30, 74, 24, 63, 82, 31, 12, 81, 29, 17, 31, 80, 29, 10, 82, 15, 17, 0.01, 
      28, 68, 91, 19, 9, 24, 111, 11, 26, 11, 40, 94, 23, 12, 30, 16, 20, 72, 98, 17, 23, 19, 73, 88, 
      92, 81, 69, 39]

t2 = [114, 9908, 191, 801, 1140, 941, 194, 4113, 22, 1111, 1172, 212, 4229, 257, 221, 2283, 79, 4022, 52,
      295, 727, 21, 
      628, 7118, 611, 719, 991, 341, 811, 831, 616, 815, 709, 754, 8663, 252, 750, 526, 480, 292, 598, 337,
      233, 739, 333, 283, 372, 321, 379, 987]


t3 =  [6, 7, 981, 80, 40, 949, 94, 436, 22, 41, 7512, 21, 49, 57, 21, 813, 79, 40, 52, 95, 77, 8, 
      68, 1, 61, 79, 99, 34, 81, 81, 66, 81, 870, 74, 83, 22, 700, 26, 400, 92, 58, 317, 23, 790, 33, 28, 
      72, 21, 79, .20]
print(len(t1), len(t2), len(t3))

# df1.loc[:,"test1"] , df1.loc[:,"test2"], df1.loc[:,"test3"] = t1, t2, t3
# df1.head(5)

50 50 50


In [2]:
# df1 = remove_outlier(df1)
# df1.reset_index(drop = True)

In [ ]:
lasso = Lasso()

# list of alphas to tune
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

# cross validation
folds = 5

# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train, y_train) 

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results.head()

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')
plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

In [ ]:
#get all categorical columns only
# creating dummy variables for categorical variables

# subset all categorical variables
cars_categorical = X.select_dtypes(include=['object'])
cars_categorical.head()


In [ ]:
df2_categorical_drop.columns.to_list()
or 
list(df2_categorical_drop.columns)
or
list(df2_categorical_drop.iloc[:,0:].columns)

# NLP

In [2]:

sentences = [
    'i love my dog',
    'I, love my cat',
    'You love my dog!',
    "I am running with dog"
]

tokenizer = Tokenizer(num_words = 1)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'cat': 5, 'you': 6, 'am': 7, 'running': 8, 'with': 9}


In [ ]:
#seeking only the numeric features from the data or selecting only categorical column
numeric_features = df2.select_dtypes(include = [np.number] or ["object"] or [np.float])

df['ExterQual'] = df.ExterQual.map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

# Generic Funtions

# <font color='green'> Checking : How much Missing values present in percentage

In [3]:
def get_percentage_miss_value(dataset):
    higher_miss_value_column = []
    miss_threshold_value = 50
    
    for i in dataset.columns:
        if dataset[i].isna().sum() > 1: 
            perectange_val = (dataset[i].isna().sum() / len(dataset)) * 100
            print("Column-> " , i, ", total no of missing value : ",dataset[i].isna().sum() , " & :         ", round(perectange_val,2) ," %")
                
            if(perectange_val > miss_threshold_value):
                higher_miss_value_column.append(i)
            
    print("\n\n\n")
    
    if higher_miss_value_column:
        print("Higher Missing values in Columns for Delete : ", higher_miss_value_column)
    else:
        print("There are no Higher Column Missing values in Dataset")
        

def check_cloumn_details_type_categorical(dataset):
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            
            print("*"*100)
            print("\n")

def check_cloumn_details_type_numberical(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "int"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            
            
def check_cloumn_details_type_float(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "float"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            
def visualize_categorical_values(dataset):
    no_of_columns = 4
    no_of_rows = 4
    
    columns_object_type = [i for i in dataset.columns  if dataset[i].dtype == "object"]
    total_rows = (len(columns_object_type) // no_of_rows ) + 1
    
    f, axes = plt.subplots(total_rows, no_of_columns, figsize=(18,24))

    for ind, val in enumerate(columns_object_type):
        sns.countplot(df[val] , ax = axes[ind // no_of_rows , ind %no_of_columns ])
    plt.show()   
    
def check_skewness_numerical(dataset):
    #analysing the distribution of sale price
    print('skew is', dataset.SalePrice.skew())   
    plt.hist(dataset['SalePrice'], color= 'b')

    plt.title('Distribution of sales price of houses', fontsize = 24)
    plt.ylabel('observation', fontsize = 20)
    plt.xlabel('sales price', fontsize = 20)

    plt.show()
    
def skewness_after_log_transform(dataset):
    #log transforming sale price to transform it into gaussian distribution
    target = np.log(df2.SalePrice)
    print('skew is', target.skew())
    plt.hist(target, color= 'b')

    plt.title('Distribution of sales price of houses', fontsize = 24)
    plt.ylabel('observation', fontsize = 20)
    plt.xlabel('sales price', fontsize = 20)

    plt.show()
    
# Since pairplots for all numeric values together are not clear we can make groups ,do plot with price & analyse

def pair_plot(list_4_numberical_values):
    sns.pairplot(df2, x_vars= list_4_numberical_values, y_vars='SalePrice',size=4, kind='scatter')
    plt.show()
    
def get_column_integer_float_list(dataset):
    numberical_int_columns = []
    for i in df2.columns:
        if df2[i].dtype == "int":
            numberical_int_columns.append(i)
        
    numberical_float_columns = []
    for i in df2.columns:
        if df2[i].dtype == "float":
            numberical_float_columns.append(i)
            
    return numberical_int_columns, numberical_float_columns

# pair_plot(numberical_int_columns[0:3] or numberical_float_columns[0:3])

In [4]:
def drop_outliers(dataset, columns_name):
    list = []
    for chk in columns_name:
        
#         plt.boxplot(dataset[chk]) # please with graph & it is optional
        
        Q1 = dataset[chk].quantile(.25)
        Q3 = dataset[chk].quantile(.75)
        IQR = Q3-Q1
        dataset =  dataset[(dataset[chk] >= (Q1-(1.5*IQR))) & (dataset[chk] <= (Q3+(1.5*IQR)))] 
    return dataset




def corr_metrix():
    corr = df2.corr()
    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap='RdYlGn')

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});
    ax.tick_params(labelsize=20)



def corr_2_more_visualize(dataset):
    corr = dataset.corr()
    
    f, ax = plt.subplots(figsize=(20, 9))
    sns.heatmap(corr.apply(lambda x : np.round(x,2)), 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,annot=True,cmap='RdYlGn', annot_kws={"size": 15})
    ax.tick_params(labelsize=20)
    plt.show()
    
def convet_categorical_numerical_use_get_dummy(dataset):
    df2_categorical = dataset.select_dtypes(include =  ["object"] )
    df_dummies = pd.get_dummies(df2_categorical,drop_first=True)
    
    # drop categorical variables 
    dataset = dataset.drop(columns=list(df2_categorical.columns), axis=1)
    
    # concat dummy variables with X
    dataset = pd.concat([dataset, df_dummies], axis=1)
    
    return dataset

def reg_evaluation(y_actual, y_pred, data_type):
    print("Model Evaluation Dataset Type is  : ", data_type)
    print("\n")
    
    print("Accuracy  :  ", r2_score(y_actual, y_pred))
    print("MAE       :  ", mean_absolute_error(y_actual, y_pred))
#     print("MSLE      :  ", mean_squared_log_error(y_actual, y_pred))
    print("MSE       :  ", mean_squared_error(y_actual, y_pred))
    print("RMSE      :  ", np.sqrt(mean_squared_error(y_actual, y_pred)))
    
def linear_regression_train_test():
    #Linear regression with L2 regularization
    for i in range(-2, 3):
        alpha = 10**i
        rm = Ridge(alpha = alpha)
        ridge_model = rm.fit(X_train, y_train)
        preds_ridge = ridge_model.predict(X_test)
    
        plt.scatter(preds_ridge, y_test, alpha= 0.75, c= 'b')
        plt.xlabel('Predicted price')
        plt.ylabel('Actual price')
        plt.title('Ridge redularization with alpha {}'.format(alpha))
        overlay = 'R square: {} \nRMSE: {}'.format(ridge_model.score(X_test, y_test), mean_squared_error(y_test, preds_ridge))
        plt.annotate(s = overlay, xy = (12.1, 10.6), size = 'x-large')
        plt.show()
        
def actual_predict_visualization(actual_values, predict_values):
    plt.scatter(actual_values, predict_values, alpha= 0.75, color = 'b')

    plt.xlabel('Predicted price')
    plt.ylabel('Actual price')
    plt.title('Regression Model')
    plt.show()

In [5]:
rows=[]
def addRandomStateForAlgorithm(x,y,names,algorithms,columns_name,random_state_list):    
    for j in range(len(algorithms)):
        model = algorithms[j]
        for i in random_state_list:
            x_train, x_test , y_train , y_test = train_test_split(x ,y , test_size = 0.25 , random_state = i)
            model.fit(x_train,y_train)
            pred_test = model.predict(x_test)
            row = [names[j],i,r2_score (y_test,pred_test)]
            rows.append(row)
    models_df = pd.DataFrame(rows)   
    models_df.columns = columns_name
    print(models_df)
    

    
    
def mean_absolute_percentage_error(y_true, y_pred) :
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    return np.mean(np.abs( (y_true - y_pred) / 100) ) * 100


def root_mean_sequare_error(y_true, y_pred) : 
    mse = mean_squared_error(y_true,  y_pred)  
    return np.sqrt(mse)
   

rows=[]

def addRandomStateForAlgorithm(x,y,names,algorithms,columns_name,random_state_list):    
    for j in range(len(algorithms)):
        model = algorithms[j]
        for i in random_state_list:
            
            x_train, x_test , y_train , y_test = train_test_split(x ,y , test_size = 0.30 , random_state = i)
            
            model.fit(x_train,y_train)
            
            y_pred_train = model.predict(x_train)
            y_pred = model.predict(x_test)
            
            train_acc = r2_score(y_train, y_pred_train)
            train_acc = round(train_acc, 2) * 100
            
            test_acc = r2_score(y_test, y_pred)
            test_acc = round(test_acc, 2) * 100
            
            mape = mean_absolute_percentage_error(y_test, y_pred)
            mape = round(mape, 2)
            
            mae = mean_absolute_error(y_test, y_pred)   
            mae = round(mae, 2)
            
            mse = mean_squared_error(y_test, y_pred)
            mse = round(mse, 2)
            
            rmse = root_mean_sequare_error(y_test, y_pred)
            rmse = round(rmse, 2)

            row = [names[j],   i,   train_acc, test_acc,  mae,    mse]
    
            rows.append(row)
            
    models_df = pd.DataFrame(rows) 
    
    models_df.columns = columns_name
    print(models_df)
    
names_regression = [ "Lasso", "Ridge"]
algorithms = [ Lasso(), Ridge(alpha=ridge_alpha)]

columns_name = ["Model",    "Random_state",   'Train_acc',     "Test_acc",   "MAE",   "MSE"]

random_state_list_up_to_10 = [0,1,2,3,4,5,6,7,8,9,10]


addRandomStateForAlgorithm(X,y,names_regression,algorithms,columns_name,random_state_list_up_to_10)